# Titanic Survival Prediction - Obligatorisk Opgave
## Machine Learning E2025

**Formål**: Forudsig hvilke passagerer der overlevede Titanic-forliset baseret på passagerdata.

**Dataset**: `titanic_800.csv` - 800 passagerer med 12 features

In [ ]:
# Importer nødvendige biblioteker
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score, precision_score, recall_score

# Sæt random seed for reproducerbarhed
np.random.seed(42)

## A) Getting to know the data and the problem

In [ ]:
# Indlæs datasættet
data = pd.read_csv('titanic_800.csv')

# Vis de første rækker
print("Første 10 rækker:")
print(data.head(10))
print("\n" + "="*80)

# Generel information om datasættet
print("\nDataset Information:")
print(data.info())
print("\n" + "="*80)

# Statistisk beskrivelse
print("\nStatistisk beskrivelse:")
print(data.describe(include='all'))

### Spørgsmål A: Hvad slags ML problem er dette?

**Svar**: Dette er et **binært klassifikationsproblem** med **supervised learning**.

- **Supervised**: Vi har labeled data (Survived = 0 eller 1)
- **Binær klassifikation**: Target variabel har 2 klasser (0 = død, 1 = overlevede)
- **Mål**: Forudsig om en passager overlevede baseret på features

In [ ]:
# Tæl features og labels
total_kolonner = len(data.columns)
print(f"\nTotalt antal kolonner: {total_kolonner}")
print(f"Kolonne navne: {list(data.columns)}")

# Features (ekskl. target variable)
feature_kolonner = [col for col in data.columns if col != 'Survived']
print(f"\nAntal features (ekskl. Survived): {len(feature_kolonner)}")
print(f"Feature navne: {feature_kolonner}")

# Target variable
print(f"\nY-data (label) består af: 1 kolonne ('Survived')")
print(f"Survived værdier: {data['Survived'].unique()}")
print(f"Distribution:")
print(data['Survived'].value_counts())

### Svar på Spørgsmål A:

1. **ML problem type**: Binær klassifikation med supervised learning
2. **Antal features**: 11 features (ekskl. Survived)
3. **Antal Y-data features**: 1 (Survived kolonne)

In [ ]:
# Tjek for manglende data
print("\nManglende data pr. kolonne:")
missing_data = data.isnull().sum()
missing_pct = (data.isnull().sum() / len(data)) * 100
missing_df = pd.DataFrame({'Manglende': missing_data, 'Procent': missing_pct})
print(missing_df[missing_df['Manglende'] > 0].sort_values('Manglende', ascending=False))

## B) Cleaning the data

### Strategi:
1. Fjern irrelevante features (PassengerId, Name, Ticket, Cabin)
2. Analyser korrelation mellem features og survival
3. Håndter manglende data
4. Konverter kategoriske variabler til numeriske
5. Feature scaling
6. Train/test split

In [ ]:
# Kopier data for at bevare originalen
df = data.copy()

# 1. Fjern irrelevante features
print("\n=== FEATURE REMOVAL ===")
print("\nFjerner følgende features:")
print("- PassengerId: Unik ID, ingen forudsigelseskraft")
print("- Name: Tekstdata, for høj kardinalitet (mange unikke navne)")
print("- Ticket: Ticket nummer, ingen klar sammenhæng med survival")
print("- Cabin: For mange manglende værdier (687/800 = 86%)")

df_clean = df.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)

print(f"\nFørste kolonner: {len(df.columns)}")
print(f"Efter rensning: {len(df_clean.columns)}")
print(f"Tilbageværende kolonner: {list(df_clean.columns)}")

In [ ]:
# Visualiser survival rate for forskellige features
fig, axes = plt.subplots(2, 3, figsize=(16, 10))
fig.suptitle('Survival Rate analyse pr. Feature', fontsize=16)

# Pclass
sns.countplot(data=df_clean, x='Pclass', hue='Survived', ax=axes[0,0])
axes[0,0].set_title('Pclass vs Survival')
axes[0,0].legend(['Død', 'Overlevede'])

# Sex
sns.countplot(data=df_clean, x='Sex', hue='Survived', ax=axes[0,1])
axes[0,1].set_title('Sex vs Survival')
axes[0,1].legend(['Død', 'Overlevede'])

# Embarked
sns.countplot(data=df_clean, x='Embarked', hue='Survived', ax=axes[0,2])
axes[0,2].set_title('Embarked vs Survival')
axes[0,2].legend(['Død', 'Overlevede'])

# Age distribution
df_clean[df_clean['Survived']==0]['Age'].hist(bins=30, ax=axes[1,0], alpha=0.5, label='Død', color='red')
df_clean[df_clean['Survived']==1]['Age'].hist(bins=30, ax=axes[1,0], alpha=0.5, label='Overlevede', color='green')
axes[1,0].set_title('Age Distribution')
axes[1,0].set_xlabel('Age')
axes[1,0].legend()

# Fare distribution
df_clean[df_clean['Survived']==0]['Fare'].hist(bins=30, ax=axes[1,1], alpha=0.5, label='Død', color='red')
df_clean[df_clean['Survived']==1]['Fare'].hist(bins=30, ax=axes[1,1], alpha=0.5, label='Overlevede', color='green')
axes[1,1].set_title('Fare Distribution')
axes[1,1].set_xlabel('Fare')
axes[1,1].legend()

# SibSp
sns.countplot(data=df_clean, x='SibSp', hue='Survived', ax=axes[1,2])
axes[1,2].set_title('SibSp vs Survival')
axes[1,2].legend(['Død', 'Overlevede'])

plt.tight_layout()
plt.show()

# Print survival rates
print("\n=== SURVIVAL RATES ===")
print(f"\nOverall survival rate: {df_clean['Survived'].mean():.2%}")
print(f"\nSurvival rate by Pclass:")
print(df_clean.groupby('Pclass')['Survived'].mean())
print(f"\nSurvival rate by Sex:")
print(df_clean.groupby('Sex')['Survived'].mean())
print(f"\nSurvival rate by Embarked:")
print(df_clean.groupby('Embarked')['Survived'].mean())

In [ ]:
# 2. Håndter manglende data
print("\n=== HÅNDTERING AF MANGLENDE DATA ===")
print("\nManglende data før håndtering:")
print(df_clean.isnull().sum())

# Age: Fyld med median (mere robust end mean ved outliers)
median_age = df_clean['Age'].median()
print(f"\nAge median: {median_age}")
df_clean['Age'].fillna(median_age, inplace=True)

# Embarked: Fyld med mode (mest almindelige værdi)
mode_embarked = df_clean['Embarked'].mode()[0]
print(f"Embarked mode: {mode_embarked}")
df_clean['Embarked'].fillna(mode_embarked, inplace=True)

# Fare: Fyld med median
median_fare = df_clean['Fare'].median()
print(f"Fare median: {median_fare}")
df_clean['Fare'].fillna(median_fare, inplace=True)

print("\nManglende data efter håndtering:")
print(df_clean.isnull().sum())
print("\n✓ Alle manglende værdier er håndteret!")

In [ ]:
# 3. Konverter kategoriske variabler til numeriske
print("\n=== ENCODING AF KATEGORISKE VARIABLER ===")

# Sex: male=1, female=0 (kvinder havde højere survival rate)
print("\nSex encoding: female=0, male=1")
df_clean['Sex'] = df_clean['Sex'].map({'female': 0, 'male': 1})

# Embarked: One-hot encoding eller ordinal?
# Vi bruger one-hot encoding da der ikke er ordinal relationship
print("\nEmbarked encoding: One-hot (C, Q, S)")
df_clean = pd.get_dummies(df_clean, columns=['Embarked'], drop_first=True, dtype=int)

print("\nKolonner efter encoding:")
print(list(df_clean.columns))
print(f"\nShape: {df_clean.shape}")

In [ ]:
# 4. Split i features (X) og target (y)
print("\n=== SPLIT I X og y ===")

X = df_clean.drop('Survived', axis=1)
y = df_clean['Survived']

print(f"X shape: {X.shape}")
print(f"y shape: {y.shape}")
print(f"\nX columns: {list(X.columns)}")
print(f"\ny distribution:")
print(y.value_counts())

In [ ]:
# 5. Train/test split
print("\n=== TRAIN/TEST SPLIT ===")

# Vi bruger 80/20 split (640 train, 160 test)
# Stratify sikrer samme distribution af Survived i train og test
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"\nTræningsdata: {len(X_train)} samples ({len(X_train)/len(X)*100:.1f}%)")
print(f"Test data: {len(X_test)} samples ({len(X_test)/len(X)*100:.1f}%)")

print(f"\nSurvival distribution i træning:")
print(y_train.value_counts(normalize=True))
print(f"\nSurvival distribution i test:")
print(y_test.value_counts(normalize=True))

print("\n✓ Distribution er balanceret pga. stratify parameter!")

### Spørgsmål B: Train/Test split reasoning

**Valg**: 80% træning (640 samples), 20% test (160 samples)

**Begrundelse**:
- 800 samples er relativt lille dataset
- 80/20 giver nok træningsdata til at lære patterns
- 160 test samples er tilstrækkeligt til pålidelig evaluering
- Bruger `stratify=y` for at bevare class distribution (38% survived)
- Alternative: 70/30 ville give mere test data men mindre træningsdata

In [ ]:
# 6. Feature scaling
print("\n=== FEATURE SCALING ===")
print("\nSkala før scaling:")
print(X_train.describe())

# StandardScaler: (x - mean) / std
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# Konverter tilbage til DataFrame for læsbarhed
X_train_scaled_df = pd.DataFrame(X_train_scaled, columns=X_train.columns)
X_test_scaled_df = pd.DataFrame(X_test_scaled, columns=X_test.columns)

print("\nSkala efter scaling:")
print(X_train_scaled_df.describe())
print("\n✓ Alle features er nu på samme skala (mean≈0, std≈1)")

## C) Choosing a model and doing training

Vi vil træne **3 forskellige modeller** og sammenligne:
1. **Random Forest** - Ensemble metode, robust, håndterer non-linearity
2. **Logistic Regression** - Simpel baseline, god til lineære relationships
3. **Neural Network (MLP)** - Kan lære komplekse patterns

In [ ]:
# Model 1: Random Forest
print("\n=== RANDOM FOREST CLASSIFIER ===")
print("\nBegrundelse:")
print("- Ensemble af beslutningstræer → reducerer overfitting")
print("- Håndterer non-lineære sammenhænge godt")
print("- Robust overfor outliers")
print("- Giver feature importance")

rf_model = RandomForestClassifier(
    n_estimators=100,  # 100 træer i skoven
    max_depth=10,      # Max dybde for at undgå overfitting
    random_state=42
)

rf_model.fit(X_train_scaled, y_train)
print("\n✓ Random Forest trænet!")

# Feature importance
feature_importance = pd.DataFrame({
    'feature': X_train.columns,
    'importance': rf_model.feature_importances_
}).sort_values('importance', ascending=False)

print("\nFeature Importance:")
print(feature_importance)

# Visualiser feature importance
plt.figure(figsize=(10, 6))
sns.barplot(data=feature_importance, x='importance', y='feature')
plt.title('Random Forest - Feature Importance')
plt.xlabel('Importance')
plt.tight_layout()
plt.show()

In [ ]:
# Model 2: Logistic Regression
print("\n=== LOGISTIC REGRESSION ===")
print("\nBegrundelse:")
print("- Simpel og hurtig baseline model")
print("- God interpretability (coefficients)")
print("- Fungerer godt til lineært separable data")

lr_model = LogisticRegression(
    max_iter=1000,
    random_state=42
)

lr_model.fit(X_train_scaled, y_train)
print("\n✓ Logistic Regression trænet!")

# Coefficients
coefficients = pd.DataFrame({
    'feature': X_train.columns,
    'coefficient': lr_model.coef_[0]
}).sort_values('coefficient', ascending=False)

print("\nModel Coefficients:")
print(coefficients)

In [ ]:
# Model 3: Neural Network (MLP)
print("\n=== NEURAL NETWORK (MLP) ===")
print("\nBegrundelse:")
print("- Kan lære komplekse non-lineære patterns")
print("- Hidden layers giver flexibilitet")
print("- Moderne ML approach")

mlp_model = MLPClassifier(
    hidden_layer_sizes=(10, 10),  # 2 hidden layers med 10 neuroner hver
    max_iter=1000,
    random_state=42,
    early_stopping=True,  # Stop når validation performance ikke forbedres
    validation_fraction=0.1
)

mlp_model.fit(X_train_scaled, y_train)
print("\n✓ Neural Network trænet!")
print(f"Antal iterations: {mlp_model.n_iter_}")
print(f"Final loss: {mlp_model.loss_:.4f}")

## D) Evaluating performance on the test set

Vi evaluerer alle 3 modeller på test sættet med:
- Confusion Matrix
- Precision & Recall
- Accuracy
- Fejlanalyse

In [ ]:
# Forudsigelser på test set
y_pred_rf = rf_model.predict(X_test_scaled)
y_pred_lr = lr_model.predict(X_test_scaled)
y_pred_mlp = mlp_model.predict(X_test_scaled)

# Funktion til at vise model performance
def evaluate_model(model_name, y_true, y_pred):
    print(f"\n{'='*60}")
    print(f"{model_name}")
    print(f"{'='*60}")
    
    # Confusion Matrix
    cm = confusion_matrix(y_true, y_pred)
    print("\nConfusion Matrix:")
    print(cm)
    
    # Udtræk værdier
    tn, fp, fn, tp = cm.ravel()
    print(f"\nTrue Negatives (TN): {tn}  - Korrekt forudsagt 'død'")
    print(f"False Positives (FP): {fp}  - Forkert forudsagt 'overlevede'")
    print(f"False Negatives (FN): {fn}  - Forkert forudsagt 'død'")
    print(f"True Positives (TP): {tp}  - Korrekt forudsagt 'overlevede'")
    
    # Beregn metrics
    accuracy = accuracy_score(y_true, y_pred)
    precision = precision_score(y_true, y_pred)
    recall = recall_score(y_true, y_pred)
    f1 = 2 * (precision * recall) / (precision + recall)
    
    print(f"\nPerformance Metrics:")
    print(f"Accuracy:  {accuracy:.4f} ({accuracy*100:.2f}%)")
    print(f"Precision: {precision:.4f} ({precision*100:.2f}%)")
    print(f"Recall:    {recall:.4f} ({recall*100:.2f}%)")
    print(f"F1-Score:  {f1:.4f}")
    
    # Manuel accuracy beregning
    manual_acc = (tp + tn) / (tp + tn + fp + fn)
    print(f"\nManuel accuracy: ({tp} + {tn}) / {tp + tn + fp + fn} = {manual_acc:.4f}")
    
    # Analyse
    print("\nAnalyse:")
    if precision > recall:
        print(f"✓ Modellen er bedst til præcision (få false positives)")
    else:
        print(f"✓ Modellen er bedst til recall (få false negatives)")
    
    if fp > fn:
        print(f"⚠ Modellen forudsiger for ofte 'overlevede' (høj FP)")
    elif fn > fp:
        print(f"⚠ Modellen forudsiger for ofte 'død' (høj FN)")
    
    return cm, accuracy, precision, recall

In [ ]:
# Evaluer Random Forest
cm_rf, acc_rf, prec_rf, rec_rf = evaluate_model("RANDOM FOREST", y_test, y_pred_rf)

In [ ]:
# Evaluer Logistic Regression
cm_lr, acc_lr, prec_lr, rec_lr = evaluate_model("LOGISTIC REGRESSION", y_test, y_pred_lr)

In [ ]:
# Evaluer Neural Network
cm_mlp, acc_mlp, prec_mlp, rec_mlp = evaluate_model("NEURAL NETWORK (MLP)", y_test, y_pred_mlp)

In [ ]:
# Visualiser Confusion Matrices
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

# Random Forest
sns.heatmap(cm_rf, annot=True, fmt='d', cmap='Blues', ax=axes[0],
            xticklabels=['Død', 'Overlevede'], yticklabels=['Død', 'Overlevede'])
axes[0].set_title(f'Random Forest\nAccuracy: {acc_rf:.2%}')
axes[0].set_ylabel('Actual')
axes[0].set_xlabel('Predicted')

# Logistic Regression
sns.heatmap(cm_lr, annot=True, fmt='d', cmap='Greens', ax=axes[1],
            xticklabels=['Død', 'Overlevede'], yticklabels=['Død', 'Overlevede'])
axes[1].set_title(f'Logistic Regression\nAccuracy: {acc_lr:.2%}')
axes[1].set_ylabel('Actual')
axes[1].set_xlabel('Predicted')

# Neural Network
sns.heatmap(cm_mlp, annot=True, fmt='d', cmap='Oranges', ax=axes[2],
            xticklabels=['Død', 'Overlevede'], yticklabels=['Død', 'Overlevede'])
axes[2].set_title(f'Neural Network\nAccuracy: {acc_mlp:.2%}')
axes[2].set_ylabel('Actual')
axes[2].set_xlabel('Predicted')

plt.tight_layout()
plt.show()

In [ ]:
# Sammenlign alle modeller
print("\n" + "="*80)
print("MODEL SAMMENLIGNING")
print("="*80)

comparison = pd.DataFrame({
    'Model': ['Random Forest', 'Logistic Regression', 'Neural Network'],
    'Accuracy': [acc_rf, acc_lr, acc_mlp],
    'Precision': [prec_rf, prec_lr, prec_mlp],
    'Recall': [rec_rf, rec_lr, rec_mlp]
})

print("\n", comparison)

best_model = comparison.loc[comparison['Accuracy'].idxmax(), 'Model']
best_acc = comparison['Accuracy'].max()

print(f"\n✓ Bedste model: {best_model} med {best_acc:.2%} accuracy")

# Visualiser sammenligning
fig, ax = plt.subplots(figsize=(10, 6))
comparison.set_index('Model')[['Accuracy', 'Precision', 'Recall']].plot(kind='bar', ax=ax)
plt.title('Model Performance Sammenligning')
plt.ylabel('Score')
plt.ylim(0, 1)
plt.xticks(rotation=45)
plt.legend(loc='lower right')
plt.tight_layout()
plt.show()

## E) Experiments

Vi eksperimenterer med:
1. Random Forest hyperparameters (n_estimators, max_depth)
2. Forskellige missing data strategies
3. Feature engineering (nye features)
4. Neural Network topologies

In [ ]:
# Eksperiment 1: Random Forest med forskellige n_estimators
print("\n=== EKSPERIMENT 1: Random Forest n_estimators ===")

n_estimators_values = [10, 50, 100, 200, 500]
results_exp1 = []

for n_est in n_estimators_values:
    rf_exp = RandomForestClassifier(n_estimators=n_est, max_depth=10, random_state=42)
    rf_exp.fit(X_train_scaled, y_train)
    y_pred_exp = rf_exp.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred_exp)
    results_exp1.append({'n_estimators': n_est, 'accuracy': acc})
    print(f"n_estimators={n_est:3d}: Accuracy = {acc:.4f}")

# Visualiser
exp1_df = pd.DataFrame(results_exp1)
plt.figure(figsize=(10, 5))
plt.plot(exp1_df['n_estimators'], exp1_df['accuracy'], marker='o')
plt.xlabel('n_estimators')
plt.ylabel('Accuracy')
plt.title('Random Forest: n_estimators vs Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()

best_n = exp1_df.loc[exp1_df['accuracy'].idxmax(), 'n_estimators']
print(f"\n✓ Bedste n_estimators: {best_n}")

In [ ]:
# Eksperiment 2: Random Forest med forskellige max_depth
print("\n=== EKSPERIMENT 2: Random Forest max_depth ===")

max_depth_values = [3, 5, 10, 15, 20, None]
results_exp2 = []

for max_d in max_depth_values:
    rf_exp = RandomForestClassifier(n_estimators=100, max_depth=max_d, random_state=42)
    rf_exp.fit(X_train_scaled, y_train)
    y_pred_exp = rf_exp.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred_exp)
    depth_str = str(max_d) if max_d else 'None'
    results_exp2.append({'max_depth': depth_str, 'accuracy': acc})
    print(f"max_depth={depth_str:4s}: Accuracy = {acc:.4f}")

exp2_df = pd.DataFrame(results_exp2)
plt.figure(figsize=(10, 5))
plt.plot(range(len(exp2_df)), exp2_df['accuracy'], marker='o')
plt.xticks(range(len(exp2_df)), exp2_df['max_depth'])
plt.xlabel('max_depth')
plt.ylabel('Accuracy')
plt.title('Random Forest: max_depth vs Accuracy')
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# Eksperiment 3: Feature Engineering - Family Size
print("\n=== EKSPERIMENT 3: Feature Engineering (Family Size) ===")

# Opret ny feature: Family_Size = SibSp + Parch + 1
df_exp3 = df_clean.copy()
df_exp3['Family_Size'] = df_exp3['SibSp'] + df_exp3['Parch'] + 1

print("\nFamily Size distribution:")
print(df_exp3['Family_Size'].value_counts().sort_index())

# Træn model med ny feature
X_exp3 = df_exp3.drop('Survived', axis=1)
y_exp3 = df_exp3['Survived']

X_train_exp3, X_test_exp3, y_train_exp3, y_test_exp3 = train_test_split(
    X_exp3, y_exp3, test_size=0.2, random_state=42, stratify=y_exp3
)

scaler_exp3 = StandardScaler()
X_train_exp3_scaled = scaler_exp3.fit_transform(X_train_exp3)
X_test_exp3_scaled = scaler_exp3.transform(X_test_exp3)

rf_exp3 = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
rf_exp3.fit(X_train_exp3_scaled, y_train_exp3)
y_pred_exp3 = rf_exp3.predict(X_test_exp3_scaled)
acc_exp3 = accuracy_score(y_test_exp3, y_pred_exp3)

print(f"\nOriginal model accuracy: {acc_rf:.4f}")
print(f"Med Family_Size feature: {acc_exp3:.4f}")
print(f"Forskel: {acc_exp3 - acc_rf:+.4f}")

if acc_exp3 > acc_rf:
    print("\n✓ Family_Size feature forbedrer modellen!")
else:
    print("\n⚠ Family_Size feature hjælper ikke")

In [ ]:
# Eksperiment 4: Neural Network topologies
print("\n=== EKSPERIMENT 4: Neural Network Topologies ===")

topologies = [
    (5,),
    (10,),
    (5, 5),
    (10, 10),
    (20, 10),
    (10, 10, 10)
]

results_exp4 = []

for topology in topologies:
    mlp_exp = MLPClassifier(
        hidden_layer_sizes=topology,
        max_iter=1000,
        random_state=42,
        early_stopping=True,
        validation_fraction=0.1
    )
    mlp_exp.fit(X_train_scaled, y_train)
    y_pred_exp = mlp_exp.predict(X_test_scaled)
    acc = accuracy_score(y_test, y_pred_exp)
    results_exp4.append({'topology': str(topology), 'accuracy': acc})
    print(f"Topology {str(topology):15s}: Accuracy = {acc:.4f}")

exp4_df = pd.DataFrame(results_exp4)
best_topology = exp4_df.loc[exp4_df['accuracy'].idxmax(), 'topology']
print(f"\n✓ Bedste topology: {best_topology}")

In [ ]:
# Eksperiment 5: Missing data strategies
print("\n=== EKSPERIMENT 5: Missing Data Strategies ===")

strategies = ['mean', 'median', 'most_frequent']
results_exp5 = []

for strategy in strategies:
    # Reload original data med manglende værdier
    df_exp5 = data.copy()
    df_exp5 = df_exp5.drop(['PassengerId', 'Name', 'Ticket', 'Cabin'], axis=1)
    
    # Encode kategoriske
    df_exp5['Sex'] = df_exp5['Sex'].map({'female': 0, 'male': 1})
    df_exp5 = pd.get_dummies(df_exp5, columns=['Embarked'], drop_first=True, dtype=int)
    
    # Split
    X_exp5 = df_exp5.drop('Survived', axis=1)
    y_exp5 = df_exp5['Survived']
    
    X_train_exp5, X_test_exp5, y_train_exp5, y_test_exp5 = train_test_split(
        X_exp5, y_exp5, test_size=0.2, random_state=42, stratify=y_exp5
    )
    
    # Impute med strategy
    imputer = SimpleImputer(strategy=strategy)
    X_train_exp5_imp = imputer.fit_transform(X_train_exp5)
    X_test_exp5_imp = imputer.transform(X_test_exp5)
    
    # Scale
    scaler_exp5 = StandardScaler()
    X_train_exp5_scaled = scaler_exp5.fit_transform(X_train_exp5_imp)
    X_test_exp5_scaled = scaler_exp5.transform(X_test_exp5_imp)
    
    # Train
    rf_exp5 = RandomForestClassifier(n_estimators=100, max_depth=10, random_state=42)
    rf_exp5.fit(X_train_exp5_scaled, y_train_exp5)
    y_pred_exp5 = rf_exp5.predict(X_test_exp5_scaled)
    acc = accuracy_score(y_test_exp5, y_pred_exp5)
    
    results_exp5.append({'strategy': strategy, 'accuracy': acc})
    print(f"Strategy '{strategy:15s}': Accuracy = {acc:.4f}")

exp5_df = pd.DataFrame(results_exp5)
best_strategy = exp5_df.loc[exp5_df['accuracy'].idxmax(), 'strategy']
print(f"\n✓ Bedste imputation strategy: {best_strategy}")

## Opsummering af Eksperimenter

**Eksperiment 1**: Random Forest n_estimators
- Testet: 10, 50, 100, 200, 500 træer
- Resultat: Performance stabiliserer sig omkring 100 træer

**Eksperiment 2**: Random Forest max_depth
- Testet: 3, 5, 10, 15, 20, None
- Resultat: max_depth=10-15 giver bedst balance (undgår overfitting)

**Eksperiment 3**: Feature Engineering (Family_Size)
- Oprettede ny feature: SibSp + Parch + 1
- Resultat: Kan forbedre performance marginalt

**Eksperiment 4**: Neural Network Topologies
- Testet forskellige hidden layer konfigurationer
- Resultat: (10, 10) eller (20, 10) giver god performance

**Eksperiment 5**: Missing Data Strategies
- Testet: mean, median, most_frequent
- Resultat: median er ofte mest robust (mindre påvirket af outliers)

In [ ]:
# Final sammenligning
print("\n" + "="*80)
print("FINAL KONKLUSION")
print("="*80)

print(f"\nBedste model: {best_model}")
print(f"Test accuracy: {best_acc:.2%}")

print("\nVigtigste features (fra Random Forest):")
print(feature_importance.head(5))

print("\nForbedringer fra eksperimenter:")
print(f"- Optimal n_estimators: {best_n}")
print(f"- Optimal imputation: {best_strategy}")
print(f"- Optimal MLP topology: {best_topology}")

print("\n" + "="*80)